In this notebook, we'll use a pre-trained machine learning model to generate a submission to the [BirdClef2023 competition](https://www.kaggle.com/c/birdclef-2023).  The goal of the competition is to identify Eastern African bird species by sound.

## Step 1: Imports

In [1]:
# Install All libraries
!pip install -Uq fastcore fastai fastbook
!pip install image_tabular
!pip install kaggle
! [ -e /content ] && pip install -Uqq pip fastai git+https://github.com/drscotthawley/fastproaudio.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import files, drive
drive.mount("/content/gdrive")
# Switch do DeepLearning directory
%cd /content/gdrive/MyDrive/DeepLearning

Mounted at /content/gdrive
/content/gdrive/MyDrive/DeepLearning


In [3]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import random
import glob
import matplotlib.pyplot as plt
from typing import List

import csv
import io

import os

from IPython.display import Audio

import torchaudio
import torch
import torchvision.transforms as T


# To save spectrograms as png
import skimage.io

# fastproaudio
from fastproaudio.core import *


# Fastai
import fastbook
fastbook.setup_book()
from fastbook import *
from fastcore.transform import Transform
from fastai.torch_core import TensorBase
from fastai.data.core import TensorImageBase

# Import all the vision library
from fastai.vision.all import *


In [4]:
# Setup kaggle API

# 1. Read the kaggle API token to interact with your kaggle account
# Folder containing kaggle.json for kaggle API authorization
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/DeepLearning/"

In [62]:
!ls
# Pick a path to download the dataset to:
comp = 'birdclef-2023'
path = URLs.path(comp)
path_train = Path(path/'train_audio')

abethr1  blhgon1  bswdov1  gryapa1	meypar1  refcro1  trobou1  yebduc1
afecuc1  bltori1  carcha1  gybfis1	nobfly1  refwar2  vibsta2  yebere1
afmdov1  brctch1  chewea1  gycwar3	palfly2  rerswa1  vilwea1  yesbar1
afrgrp1  brobab1  cohmar1  hamerk1	piecro1  sccsun2  whbcan1
bawman1  brrwhe3  gabgos2  kaggle.json	ratcis1  scrcha1  whctur2
beasun2  brtcha1  gnbcam2  loceag1	rebfir2  soucit1  wookin1
blaplo1  brubru1  gobbun1  lotcor1	reboxp1  strsee1  yebapa1


In [6]:
#And use the Kaggle API to download the dataset to that path, and extract it:
Path.BASE_PATH = path

from kaggle import api

if not path.exists():
    path.mkdir(parents=true)
    api.competition_download_cli(comp, path=path)
    shutil.unpack_archive(str(path/f'{comp}.zip'), str(path))

path.ls(file_type='text')

100%|██████████| 4.91G/4.91G [02:08<00:00, 41.1MB/s]


(#3) [Path('sample_submission.csv'),Path('eBird_Taxonomy_v2021.csv'),Path('train_metadata.csv')]

# Pre-generate Spectrograms and save them
Generate spectrograms over 5-seconds splits of resampled audio files


In [7]:
Path(path/'train_audio/gybfis1').ls()

(#10) [Path('train_audio/gybfis1/XC282065.ogg'),Path('train_audio/gybfis1/XC131985.ogg'),Path('train_audio/gybfis1/XC282061.ogg'),Path('train_audio/gybfis1/XC396326.ogg'),Path('train_audio/gybfis1/XC397814.ogg'),Path('train_audio/gybfis1/XC267529.ogg'),Path('train_audio/gybfis1/XC127885.ogg'),Path('train_audio/gybfis1/XC282058.ogg'),Path('train_audio/gybfis1/XC267528.ogg'),Path('train_audio/gybfis1/XC618866.ogg')]

Get the paths to all audio files

In [8]:
def get_audio_files(path):
    return get_files(path, extensions='.ogg', recurse=True)

def get_npy_files(path):
    return get_files(path, extensions='.npy', recurse=True)


In [9]:
def read_signal(audio_file_path: Path,
                  audio_split_length: float=5.,
                  audio_hop_length: float=1.,
                  sample_rate: int=32000):
    # Load audio sample from file path
    sig, rate = librosa.load(audio_file_path)    
    # Resample to match desired sample rate
    resampled_sig = librosa.resample(sig, orig_sr=rate, target_sr=sample_rate)
    return resampled_sig.astype(float, copy=False)

def split_signal(resampled_sig: np.ndarray,
                  audio_split_length: float=5.,
                  audio_hop_length: float=1.,
                  sample_rate: int=32000):
    "Split signal into chunks of desired length. Returns a list a those chunks" 

    len_chunk = int(audio_split_length*sample_rate)
    len_sig = len(resampled_sig)

    # If length of signal inferior to desired length, pad it with zeros
    if (len_sig <= len_chunk):
      res = np.zeros(len_chunk)
      res[:len_sig] = resampled_sig
      return [res]
    else:
      res = []
      # Create chunks. To avoid padding, the last chunk will start audio_split_length seconds before the signal end.
      start = 0
      hop = int(sample_rate*audio_hop_length)

      while(start+len_chunk <= len_sig):
        res.append(resampled_sig[start:start+len_chunk])
        start+=hop

    # Append last chunk
      res.append(resampled_sig[len_sig-len_chunk:])

      return res


def truncate_window(signal: np.ndarray,
                  window_size=10.,
                  sample_rate: int=32000):
    "Truncate signal within a random window of desired length" 

    length_window = int(window_size*sample_rate)
    len_signal = len(signal)
    if (len_signal>=length_window):
        start_window = random.randint(0,len_signal-length_window) 
        window = signal[start_window:start_window+length_window]
        return [window]
    else:
        window = np.zeros(length_window)
        window[0:len_signal] = signal
        return [window]


In [10]:
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def create_spectrogram(path_save, signal:np.ndarray, sample_rate:int=32000):
    spec = librosa.feature.melspectrogram(y=signal, sr=sample_rate)#, n_fft=int(duration*sample_rate))
    # save file
    np.save(path_save, spec)  


In [11]:
def save_chunk_as_file(chunk, audio_file_path: Path, sample_rate:int=32000, extension:str='.npy', idx_chunk=0, split=False):
    if split==True:
      path_save = Path(audio_file_path.parent/(audio_file_path.stem+f"-{idx_chunk}"+extension))
    else:
      path_save = Path(audio_file_path.parent/(audio_file_path.stem+extension))

    if not path_save.exists():
      create_spectrogram(path_save, chunk, sample_rate)
      print(path_save)
    else:
      print(path_save)
      print("LAST ALREADY EXISTED")
    return idx_chunk + 1

def create_spectrograms(audio_file_path: Path,
                        truncate_window_bool=False,
                        window_size=10.,
                        split=False,
                        audio_split_length: float=5.,
                        audio_hop_length: float=1.,
                        sample_rate: int=32000,
                        ax=None,
                        vmin=-70,
                        vmax=0,
                        rate=32000,
                        ctx=None):
    "Create all spectrograms for one audio file and save them in the same folder as audio files"
    # Split signal in to desired length chunks or window
    signal = read_signal(audio_file_path, audio_split_length=audio_split_length, audio_hop_length=audio_hop_length, sample_rate=sample_rate) 

    if truncate_window_bool==True:
      chunks = truncate_window(signal, window_size=window_size,sample_rate=sample_rate)

    elif split==True:
      chunks = split_signal(audio_file_path, audio_split_length=audio_split_length, audio_hop_length=audio_hop_length, sample_rate=sample_rate) 
    else:
      chunks=[signal]

    for i, chunk in enumerate(chunks):
        save_chunk_as_file(chunk, audio_file_path, idx_chunk=i)

## Call the methods and save spectrograms as .npy files

In [12]:
#for i, audio_file in enumerate(audio_files):
#  create_spectrograms(audio_file, truncate_window_bool=True)
#  print(f"\nPROCESSED AUDIO FILES : {i} / {len(audio_files)}\n") # Output progress

# TRAIN LEARNER


Extract Labels from csv file

In [63]:
# Extract labels from submission csv
labels_csv_path = path/"sample_submission.csv"

def class_names_from_csv(class_map_csv_text):
    """Returns list of class names corresponding to score vector."""
    with open(class_map_csv_text) as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        class_names = csv_reader.__next__()
        # Return all columns headers except for the first one which is "rows"
        return class_names[1:]

# Put all bird ids into labels list
labels_vocab = class_names_from_csv(labels_csv_path)

Write function to show spectrograms

In [48]:
def show_spectrogram(audio_data, ax=None, vmin=-70, vmax=0, rate=32000, ctx=None):
    # Get audio tensor (values between 0. and 1.) and convert to spectrogram DB values
    spec_db = array(audio_data[0]*255)

    # Plot Mel-spectrogram plot, from librosa documentation
    if ax is None: fig, ax = plt.subplots()
    img = librosa.display.specshow(spec_db, y_axis='mel', x_axis='time', ax=None, sr=32000, vmin=-70, vmax=0)
    ax.set(title=f'Spectrogram of bird type : {audio_data[1]}')
    plt.colorbar(img, ax=ax, format="%+2.f dB")
    if ax is None: plt.show(block=False)

# Before we can create a Transform, we need a type that knows how to show itself
# (if we want to use the show method). Here we define a TitledImage:
class TitledImage(fastuple):
    def show(self, ctx=None, **kwargs):
      show_spectrogram(self, ctx=ctx, **kwargs)

class TitledPILImage(fastuple):
    def show(self, ctx=None, **kwargs):
      self[0].show()

Write transform to feed .npy spectrogram files to learner

In [15]:
#transform_to_tensor = T.Compose([T.ToTensor()])
#transform_to_PIL = T.ToPILImage()


In [49]:
# Function to convert .npy files into tensors
def npy_to_tensor(path_file:Path):
    # Load file
    spec = np.load(path_file)
    # Convert to DB
    spec_db = librosa.power_to_db(spec, ref=np.max)
    # Convert spectrogram DB data to PyTorch Tensor
    spec_tensor = torch.from_numpy(spec_db).float().transpose(0, 1).unsqueeze(0)
    spec_tensor = spec_tensor / spec_tensor.max()

    return spec_tensor

def npy_to_PILImage(path_file:Path):
    # Load file
    spec = np.load(path_file)
    # Convert to DB
    spec_db = librosa.power_to_db(spec, ref=np.max)
    # Convert spectrogram DB data to PyTorch Tensor
    spec_tensor = torch.from_numpy(spec_db).float().transpose(0, 1).unsqueeze(0)
    spec_tensor = spec_tensor / spec_tensor.max()
    img = ToPILImage()(spec_tensor)
    return img

def open_npy(fname):

    img_array = np.load(fname, allow_pickle=True)
    img_tensor = torch.from_numpy(img_array).float().transpose(0, 1).unsqueeze(0)
    img_tensor = img_tensor / img_tensor.max()
    img = ToPILImage()(img_tensor)
    return img

class NpyTransform(ItemTransform):
    def setups(self, items):
      self.lblr = parent_label
      vals = class_names_from_csv(labels_csv_path)
      self.vocab,self.o2i = uniqueify(vals, sort=True, bidir=True)

    def encodes(self, o): 
      return (npy_to_tensor(o), self.o2i[self.lblr(o)])
    def decodes(self, x): 
      return TitledImage(x[0],self.vocab[x[1]])

def NpyBlock(vocab, o2i, lblr):
    return TransformBlock(type_tfms=NpyTransform(vocab, o2i, lblr))
 
class NpyToPILImageTransform(ItemTransform):
    def setups(self, items):
      self.lblr = parent_label
      vals = class_names_from_csv(labels_csv_path)
      self.vocab,self.o2i = uniqueify(vals, sort=True, bidir=True)

    def encodes(self, o): 
      return (npy_to_PILImage(o), self.o2i[self.lblr(o)])
    def decodes(self, x): 
      return TitledImage(x[0],self.vocab[x[1]])

def NpyBlock(vocab, o2i, lblr):
    return TransformBlock(type_tfms=NpyTransform(vocab, o2i, lblr))
 

In [59]:
path_spectrograms = "/content/gdrive/MyDrive/DeepLearning/"
items = get_npy_files(path_spectrograms)
audio_items = get_audio_files(path)
len(items), len(audio_items)

(2735, 0)

In [18]:
# Check for double occurences of name files
#for i, audio_file_path in enumerate(audio_items):
#  print(i)
#  count = 0
#  for audio_file_path2 in audio_items:
##    if audio_file_path.name == audio_file_path2.name:
 ##     count += 1
  #    if count == 2:
  #      print("DOUBLON")
  #      print(audio_file_path.name)
  #      print(audio_file_path2.name)
  #      break

In [19]:
# Save npy files to google drive
#for i, path_file in enumerate(items):
#  path_tmp = Path("/content/gdrive/MyDrive/DeepLearning/"+path_file.name)
#  shutil.copy(path_file, path_tmp) 
#  print(f'Copied Items : {i}/{len(items)}')


In [20]:
audio_items[1]

Path('train_audio/shesta1/XC209997.ogg')

In [21]:
## Add parent folders to npy files in gdrive
#path_npy_gdrive = Path("/content/gdrive/MyDrive/DeepLearning/")
#path_ogg_fastai = Path('/root/.fastai/archive/birdclef-2023/train_audio/')
#for i, path_npy in enumerate(items):
#  print(f'Processing {i+1}/{len(items)}')
#  for path_ogg in audio_items:
#    if path_npy.stem == path_ogg.stem:
#      dest = Path(path_npy_gdrive/path_ogg.parent.name/path_npy.name)
#      # Check if directory exists
#      if not dest.parent.exists():
#        dest.parent.mkdir(parents=True, exist_ok=True)
#      shutil.move(path_npy, dest)
#      break

In [22]:
!ls /content/gdrive/MyDrive/DeepLearning/soucit1


XC119402.npy  XC428700.npy  XC612385.npy  XC626072.npy	XC703347.npy
XC352785.npy  XC601840.npy  XC612386.npy  XC627235.npy
XC391601.npy  XC601841.npy  XC613507.npy  XC627238.npy
XC396039.npy  XC602932.npy  XC614372.npy  XC627239.npy
XC396388.npy  XC602933.npy  XC615098.npy  XC697678.npy


In [23]:
# Test
path_ogg = audio_items[1]
path_npy_gdrive = Path("/content/gdrive/MyDrive/DeepLearning/")
path_npy = items[1]
Path(path_npy_gdrive/path_ogg.parent.name/path_npy.name)

Path('/content/gdrive/MyDrive/DeepLearning/shesta1/XC608050.npy')

## Test transform class and functions

In [24]:
labeller = parent_label
vals = list(map(labeller, items))
vocab,o2i = uniqueify(vals, sort=True, bidir=True)
npy_transform = NpyTransform(vocab,o2i,labeller)

In [25]:
x,y = npy_transform(items[0])
x.shape, y


TypeError: ignored

In [ ]:
(x,y)

In [ ]:
dec = npy_transform.decode([x,y])
dec.show()

## Create Tfmdlists

In [50]:
path_npy_gdrive = Path("/content/gdrive/MyDrive/DeepLearning/")
items = get_npy_files(path_npy_gdrive)
items

(#2736) [Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC609125.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC608050.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC298959.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC398625.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC752954.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC593225.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC609472.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC419932.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC609128.npy'),Path('/content/gdrive/MyDrive/DeepLearning/sccsun2/XC536795.npy')...]

In [51]:
# Parameters of the TfmdLists
npy_transform = NpyTransform()
npy_transform.setup(items)
splits = RandomSplitter()(items)

npy_PIL_transform = NpyToPILImageTransform()
npy_PIL_transform.setup(items)


In [52]:
vals = class_names_from_csv(labels_csv_path)
vocab, o2i = uniqueify(vals, sort=True, bidir=True)
parent_label(items[0])
o2i[parent_label(items[0])]

191

In [53]:
tls = TfmdLists(items, [npy_PIL_transform], splits=splits)


In [55]:
# define datablock
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_x=open_npy,
                   get_y=parent_label,
                   item_tfms=[Resize((128, 626))],
                   splitter=RandomSplitter(seed=42))

In [54]:
dls = tls.dataloaders(bs=64)


Could not do one pass in your dataloader, there is something wrong in it. Please see the stack trace below:


AttributeError: ignored

In [ ]:
tls.train[0][0].shape

In [70]:
from torchvision.models import resnet18, resnet34, resnet50
opt_func = partial(Adam, lr=slice(3e-3), wd=0.01, eps=1e-8)

In [ ]:
# define learner and train model
learn = vision_learner(dls, resnet18, opt_func=opt_func,metrics=error_rate, loss_func=CrossEntropyLossFlat)


In [52]:
# Load the pre-trained ResNet34 model
learn = cnn_learner(dls, resnet34, pretrained=True, loss_func=CrossEntropyLossFlat)


# Modify the first layer to accept 2D tensors with 1 channel
learn.model[0] = nn.Conv2d(64,1, kernel_size=7, stride=2, padding=3, bias=False)

In [53]:
learn.fine_tune(4)

epoch,train_loss,valid_loss,time


ValueError: ignored

In [ ]:
# AUDIO ALERT TO BE ABLE TO DRINK COFFEE AWAY WHILE SOMETHING LONG IS RUNNING
from IPython.lib.display import Audio
import numpy as np

framerate = 4410
play_time_seconds = 3

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)

In [68]:
from fastai.vision.all import *
from pathlib import Path
from PIL import Image
from torchvision.transforms import ToPILImage

# get list of all .npy files in folder
fnames = items

# define function to open .npy file and return tensor
#def open_npy(fname):
#    img_array = np.load(fname, allow_pickle=True)
#    img = Image.fromarray(img_array)
#    return img
import pickle as pl

def open_npy(fname):

    img_array = np.load(fname, allow_pickle=True)
    img_tensor = torch.from_numpy(img_array).float().transpose(0, 1).unsqueeze(0)
    img_tensor = img_tensor / img_tensor.max()
    img = ToPILImage()(img_tensor)
    return img

# define function to get label from folder name
def get_label(fname):
    return Path(fname).parent.name

# define datablock
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_x=open_npy,
                   get_y=parent_label,
                   item_tfms=[Resize((128, 626))],
                   splitter=RandomSplitter())

# create dataloaders
dls = dblock.dataloaders(fnames, bs=32, vocab=labels_vocab)

# create learner
learn = cnn_learner(dls, resnet18, metrics=[accuracy])

# train model
learn.fine_tune(10)

/usr/local/lib/python3.9/dist-packages/fastai/vision/learner.py:288: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


epoch,train_loss,valid_loss,accuracy,time
0,5.216451,6.727895,0.065814,00:19


epoch,train_loss,valid_loss,accuracy,time
0,4.559604,3.905809,0.142596,00:21
1,4.412837,4.390405,0.106033,00:19
2,4.189184,3.695639,0.137112,00:20
3,4.010198,3.616767,0.155393,00:20
4,3.797895,3.358950,0.146252,00:20
5,3.602989,3.307191,0.202925,00:21
6,3.535150,3.231641,0.223035,00:20
7,3.456895,3.125556,0.230347,00:19
8,3.328866,3.019957,0.259598,00:22
9,3.309398,3.026070,0.252285,00:20


In [74]:
learn.fine_tune(100)

epoch,train_loss,valid_loss,accuracy,time
0,2.792795,2.888068,0.276051,00:18


epoch,train_loss,valid_loss,accuracy,time
0,2.746734,2.850829,0.281536,00:22
1,2.718324,2.834629,0.270567,00:19
2,2.691181,2.885295,0.246801,00:22
3,2.688514,2.844851,0.276051,00:19
4,2.667398,2.804482,0.276051,00:19
5,2.687994,2.784253,0.301645,00:19
6,2.631951,2.959460,0.255942,00:20
7,2.623416,2.793895,0.297989,00:20
8,2.620488,2.806731,0.297989,00:19
9,2.639583,3.162658,0.191956,00:19


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f87dd5a68b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f87dd5a68b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
  File "/usr/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 

In [71]:
learn.opt_func=opt_func
